  # This notebook will be used for Coursera Capstone Project

## Assignment week 3: Explore and cluster the neighborhoods in Toronto.

### We have webscrape the toronto website and obtain a clean dataframe

<p> Web scrapping is a technique to automatically access and extract large amount of information from a website,
to save huge amount of time and effort <p>

<p> Use the Notebook to build the code to scrape the following website 
<a href="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M">https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M</a> in order to obtain 
        the data that is in the table of postal codes and to transform the data into a pandas dataframe <p>


In [37]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from pandas.io.json import json_normalize
import json
import requests
import urllib.request
import time
import csv
from bs4 import BeautifulSoup

### enter the url of the site to scrape

In [12]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(url)

In [28]:
soup = BeautifulSoup(response.text,"html.parser")

In [73]:
#print(soup.prettify())

### Initialize the lists and use find_all method to find our table tags

In [76]:
postalCodeList = []
boroughList = []
neighborhoodList = []
table = soup.find('table').find_all('tr')
#print(table)
for row in table:
        cells = row.find_all('td')
        if(len(cells) > 0):
            postalCodeList.append(cells[0].text)
            boroughList.append(cells[1].text)
            neighborhoodList.append(cells[2].text.rstrip('\n')) #remove new line character from cell
            

            
    
 

In [75]:
# create a dataframe
toronto_data = [('PostalCode', postalCodeList),
                     ('Borough', boroughList),
                    ('Neighborhood', neighborhoodList)]
toronto_df = pd.DataFrame.from_dict(dict(toronto_data))
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


<b>Remove not assigned rows </b> <p> There are some postal codes which are not belongs to any borough.
We will remove those rows.</p>

In [68]:
toronto_df_dropna = toronto_df[toronto_df.Borough != 'Not assigned'].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


<b> Group neighborhoods by postal and borough </b>
<p>There are some neighborhoods that belongs to the same postal code and borough (Ex: M5A Downtown Toronto has 2 neighborhoods Harbourfront and Regent Park).
We will concat them into same row, seperated by a colon.<p>

In [69]:
toronto_df_grouped = toronto_df_dropna.groupby(['PostalCode','Borough'], as_index=False).agg(lambda x: ','.join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<b> Not Assigned Neighbourhood </b>
<p>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.</p>

In [70]:
na_neigh_rows = toronto_df_grouped.Neighborhood == 'Not assigned'
toronto_df_grouped.loc[na_neigh_rows, 'Neighborhood'] = toronto_df_grouped.loc[na_neigh_rows, 'Borough']
toronto_df_grouped[na_neigh_rows]

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


## A Cleaned Dataframe

In [71]:
toronto_df_cleaned = toronto_df_grouped
toronto_df_cleaned.shape

(103, 3)

### So we have 103 rows in the clean final dataframe